In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.table import Table

# Load your CSV file (update the file path accordingly)
csv_file_path = '/home/ka/Downloads/fv5.5.2/interpolated_theoretical_spectrum.csv'
data = pd.read_csv(csv_file_path)

# Assuming CSV has 'Energy' and 'Total Scattered Spectrum' columns
energy = data['Energy'].values
spectrum = data['Total Scattered Spectrum'].values

# Scale the spectrum values to 10^0 (from 10^-9)
spectrum_scaled = spectrum * 0.321 * 1e-10

# Ensure energy matches the required size (e.g., 3000 rows)
energy = energy[:3000]  # Truncate or adjust as necessary
energy_upper = energy + 0.01  # Example for upper bounds, scaled by 10%

# Ensure spectra matches the required size (e.g., 15 rows, 3000 columns)
spectrum_scaled = np.tile(spectrum_scaled[:3000], (15, 1))

# HDU 0: Primary Header
primary_header = fits.Header({
    'SIMPLE': True,
    'BITPIX': 8,
    'NAXIS': 0,
    'EXTEND': True,
    'MODLNAME': 'scatter_model',
    'MODLUNIT': 'photons/(cm^2 s)',
    'REDSHIFT': False,
    'ADDMODEL': True,
    'HDUCLASS': 'OGIP',
    'HDUCLAS1': 'XSPEC TABLE MODEL',
    'HDUVERS': '1.0.0',
})
hdu0 = fits.PrimaryHDU(header=primary_header)

# HDU 1: Parameters Table (Add any specific parameters here)
params_data = np.array([
    ('param1', 1, 0.1, 0.01, 0.0, 0.0, 0.1, 0.2, 1, 1.0)
], dtype=[
    ('NAME', 'U7'),
    ('METHOD', 'i2'),
    ('INITIAL', 'f4'),
    ('DELTA', 'f4'),
    ('MINIMUM', 'f4'),
    ('BOTTOM', 'f4'),
    ('TOP', 'f4'),
    ('MAXIMUM', 'f4'),
    ('NUMBVALS', 'i2'),
    ('VALUE', 'f4')
])
params_table = Table(params_data)
params_header = fits.Header({
    'XTENSION': 'BINTABLE',
    'EXTNAME': 'PARAMETERS',
    'NINTPARM': 1,
    'NADDPARM': 0,
    'HDUCLASS': 'OGIP',
    'HDUCLAS1': 'XSPEC TABLE MODEL',
    'HDUCLAS2': 'PARAMETERS',
    'HDUVERS': '1.0.0',
})
hdu1 = fits.BinTableHDU(params_table, header=params_header)

# HDU 2: Energies Table
energies_data = np.column_stack((energy.astype('float32'), energy_upper.astype('float32')))
energies_table = Table(energies_data, names=('ENERG_LO', 'ENERG_HI'))
energies_header = fits.Header({
    'XTENSION': 'BINTABLE',
    'EXTNAME': 'ENERGIES',
    'HDUCLASS': 'OGIP',
    'HDUCLAS1': 'XSPEC TABLE MODEL',
    'HDUCLAS2': 'ENERGIES',
    'HDUVERS': '1.0.0',
})
hdu2 = fits.BinTableHDU(energies_table, header=energies_header)

# HDU 3: Spectra Table
paramval = np.arange(15, dtype=np.float32)  # 4 bytes per value
INTPSPEC = [np.array(spectrum_scaled[i], dtype=np.float32) for i in range(15)]
spectra_table = Table(
    {'paramval': paramval, 'INTPSPEC': INTPSPEC},
    names=('paramval', 'INTPSPEC')
)
spectra_header = fits.Header({
    'XTENSION': 'BINTABLE',
    'EXTNAME': 'SPECTRA',
    'HDUCLASS': 'OGIP',
    'HDUCLAS1': 'XSPEC TABLE MODEL',
    'HDUCLAS2': 'MODEL SPECTRA',
    'HDUVERS': '1.0.0',
})
hdu3 = fits.BinTableHDU(spectra_table, header=spectra_header)

# Write the FITS file
output_fits_file = '/home/ka/Downloads/fv5.5.2/b.fits'
hdul = fits.HDUList([hdu0, hdu1, hdu2, hdu3])
hdul.writeto(output_fits_file, overwrite=True)

print(f"FITS file written to: {output_fits_file}")


FITS file written to: /home/ka/Downloads/fv5.5.2/b.fits
